In [2]:
%run constants.ipynb

In [14]:
def read_clean_data() -> tuple[pd.DataFrame, dict[str, str]]:
    # Load the arguments
    with open(ARGUMENTS_FILENAME, "r") as fid:
        arguments = json.load(fid)

    df = pd.read_csv(arguments["dataframe_outfile"])
    df["date clean"] = pd.to_datetime(df["date clean"])
    df.set_index("date clean", inplace=True)
    df.sort_index(inplace=True)
    
    return df, arguments

#### Cervical mucus charting

In [ ]:
def lookup_color(col: pd.Series) -> list[str]:
    """Return the correct background color, depending on whether it's a symbol or other text."""
    return [
        f"background-color: {cervical_mucus_colors[text]}; color: white; font-weight: bold;"
        if text in cervical_mucus_colors
        else f"background-color: white); color: black;"
        for text in col
    ]


def get_chart_with_text(valid_subset: pd.DataFrame) -> pd.DataFrame:
    table_df = valid_subset[["nth_cycle", "day of cycle", "symbol", "cervical mucus description"]].reset_index()
    table_df["date"] = table_df["date clean"].dt.strftime("%m/%d")
    n_chars = 30
    table_df["cervical mucus description"] = table_df["cervical mucus description"].str[:n_chars]
    table = table_df.melt(
        id_vars=["nth_cycle", "day of cycle"],
        value_vars=["date", "symbol", "cervical mucus description"],
    ).pivot(
        index=["nth_cycle", "variable"],
        columns="day of cycle",
        values="value",
    ).reset_index().sort_values("variable", ascending=False).sort_values("nth_cycle").fillna("")

    table.loc[table["variable"] == "cervical mucus description", "variable"] = "description"
    table["variable"] = pd.Categorical(table["variable"], ["date", "symbol", "description"])
    table.sort_values(["nth_cycle", "variable"], inplace=True)
    table.set_index(["nth_cycle", "variable"], inplace=True)
    return table


def chart_cervical_mucus(
    df: pd.DataFrame,
    with_text: bool,  # whether to include dates and descriptions
    outfile: Union[str, None] = None,
) -> pd.io.formats.style.Styler:
    # Create dataset where we have cycle data, so we can always pivot on day
    subset = df[df["day of cycle"].notnull()].copy()
    subset["day of cycle"] = subset["day of cycle"].astype(int)
    subset["nth_cycle"] = subset["nth_cycle"].astype(int)
    subset = subset[subset["valid_cycle"]]

    if with_text:
        table = get_chart_with_text(subset)
    else:
        table = subset.pivot(
            index="nth_cycle",
            columns="day of cycle",
            values="symbol",
        ).fillna("")

    styled_table = table.style.apply(lookup_color)
    styled_table = styled_table.set_properties(**{"max-width": "80px"})
    
    # Save the charting image as a PNG file
    if outfile is not None:
        dfi.export(
            styled_table,
            f"{arguments['output_directory']}/{outfile}.png",
            dpi=300,
            max_cols=-1,
        )

    return styled_table

#### Plot a categorical variable over time

In [ ]:
def plot_categorical_rate(
    metric: str,
    days_mean: int,
    possible_values: list[str],
    nan_is_no: bool = False,
):
    for possible_value in possible_values:
        df[f"{metric}_{possible_value}"] = (df[metric] == possible_value).astype(int)
        # assume no value means "no", if desired
        if nan_is_no and possible_value == "no":
            df.loc[df[metric].isnull(), f"{metric}_{possible_value}"] = 1

    rate_metrics = [
        f"{metric}_{possible_value}"
        for possible_value in possible_values
    ]

    fig, ax = plt.subplots(figsize=(12, 8), facecolor="w")
    for rate_metric in rate_metrics:
        if df[rate_metric].mean() == 0:
            continue

        plt.plot(
            df[rate_metric].rolling(f"{days_mean}D").mean() * 100,
            label=rate_metric.replace(f"{metric}_", ""),
        )
        fig.autofmt_xdate()

    plt.title(f"{metric.title()} frequency", size=14)
    plt.xlabel("Date", size=12)
    plt.ylabel(f"% of days with this type of {metric} ({days_mean}-day rolling mean)", size=12)
    plt.legend(title=metric.title())

#### Plot numeric metrics over time

In [25]:
def plot_numeric(df: pd.DataFrame, metrics: list[str], days_window: int) -> None:
    fig, ax = plt.subplots(figsize=(12, 8), facecolor="w")
    
    if len(metrics) == 1:
        metric = metrics[0]
        plt.plot(
            df[metric].rolling(f"{days_window}D").mean(),
        )
        title = metric.replace("_", " ").replace(" numeric", "")
        plt.title(title)
        plt.ylabel(f"{days_window}-day rolling mean")

    # compare several metrics
    else:
        for metric in metrics:
            plt.plot(
                df[metric].rolling(f"{days_window}D").mean(),
                label=metric.replace("_", " ").replace(" numeric", ""),
            )

        plt.title(f"{days_window}-day rolling mean", size=14)
        plt.xlabel("Date", size=12)
        plt.ylabel(f"{days_window}-day rolling mean", size=12)
        plt.legend(title="Metric")

    fig.autofmt_xdate()

#### Superimpose menstruation and ovulation

In [ ]:
def plot_superimpose_cycle(
    df: pd.DataFrame,
    metric: str,
    days_window: int,
    only_valid_cycles: bool,
) -> None:
    if only_valid_cycles:
        plot_data = df.loc[df["valid_cycle"]].copy()
    else:
        plot_data = df.copy()

    fig, ax = plt.subplots(figsize=(12, 8), facecolor="w")
    
    plt.plot(
        plot_data[metric].rolling(f"{days_window}D").mean(),
        label=metric.replace("_", " ").replace(" numeric", ""),
    )

    ## Peak day
    plotted_peak = False
    peak_days = plot_data.loc[plot_data["peak_day"] == True].index.values
    for peak_day in peak_days:
        # only make one legend label
        if plotted_peak:
            label = None
        else:
            label = "Peak Day"
            plotted_peak = True
        # approximate ovulation date, made a bit wider for visibility
        plt.axvspan(
            xmin=peak_day + np.timedelta64(-1,'D'),
            xmax=peak_day,
            color="royalblue",
            alpha=0.4,
            label=label,
        )

    ## Menstruation
    plotted_menstruation = False
    for cycle in plot_data["nth_cycle"].unique():
        if np.isnan(cycle):
            continue

        first_day_df = plot_data.loc[
            (plot_data["nth_cycle"] == cycle) &
            (plot_data["day of cycle"] == 1)
        ]

        last_day_df = plot_data.loc[
            (plot_data["nth_cycle"] == cycle) &
            plot_data["last_bleeding_day"]
        ]
        if first_day_df.empty or last_day_df.empty:
            continue
        first_day = first_day_df.index.values[0]
        last_day = last_day_df.index.values[0]

        # only make one legend label
        if plotted_menstruation:
            label = None
        else:
            label = "Menstruation"
            plotted_menstruation = True
        plt.axvspan(
            xmin=first_day,
            xmax=last_day,
            color="salmon",
            alpha=0.4,
            label=label,
        )

    fig.autofmt_xdate()

    plt.title(f"{metric} {days_window}-day rolling mean", size=14)
    plt.ylabel(f"{metric} {days_window}-day rolling mean", size=12)
    plt.legend(title="Metric/event")
    
    fig.savefig(
        f"{arguments['output_directory']}/{metric}_valid_only_{only_valid_cycles}.png",
    )

### Plot by cycle day, rather than by date

Note that metrics are standardized.

In [30]:
def get_metric_label(metric: str) -> str:
    return metric.replace("_", " ").replace(" numeric", "")

In [31]:
def plot_by_cycle_day(
    df: pd.DataFrame,
    metrics: list[str],
    days_window: int,
    standardize: bool,
) -> None:
    plot_data = df.loc[df["valid_cycle"]].copy()
    plot_data = plot_data.set_index("day of cycle").sort_index()

    fig, ax = plt.subplots(figsize=(12, 8), facecolor="w")

    for metric in metrics:
        # create a standardized metric, so we can compare many at once
        if standardize:
            plot_data[metric] = (
                (plot_data[metric] - plot_data[metric].mean())
                / (2 * plot_data[metric].std())
            )
            ylabel = f"Change in 2 SD, {days_window}-day rolling mean"
        else:
            ylabel = f"{days_window}-day rolling mean"

        # Note: confidence intervals are likely too tight b/c of high correlation among similar days of cycle
        # TODO: fit ARIMA model
        sns.lineplot(
            x=plot_data.index,
            y=plot_data[metric].rolling(window=days_window).mean(),
            ci=False,
            label=get_metric_label(metric),
        )

    plt.xlabel("Day of Cycle", size=12)
    plt.ylabel(ylabel, size=12)
    plt.title(f"By day of cycle, {days_window}-day mean", size=14)
    plt.legend(title="Metric")

In [27]:
def histogram(df: pd.DataFrame, metric: str) -> None:
    fig, ax = plt.subplots(figsize=(12, 8), facecolor="w")
    df[metric].hist(bins=20)
    metric_label = get_metric_label(metric)
    plt.xlabel(metric_label, size=12)
    plt.ylabel("Frequency", size=12)
    plt.title(metric_label, size=14)

In [76]:
def plot_by_group(
    plot_type: str,
    df: pd.DataFrame,
    x_var: str,
    y_var: str,
    start_dt: Union[datetime.datetime, None] = None,
) -> None:
    if start_dt is not None:
        plot_data = df[df.index >= start_dt].copy()
    else:
        plot_data = df.copy()
    
    fig, ax = plt.subplots(figsize=(12, 8), facecolor="w")
    if plot_type == "violin":
        sns.violinplot(
            x=plot_data[x_var],
            y=plot_data[y_var],
            color="gray",
        )
    elif plot_type == "line":
        sns.lineplot(
            x=plot_data[x_var],
            y=plot_data[y_var],
        )
    elif plot_type == "scatter":
        sns.scatterplot(
            x=plot_data[x_var],
            y=plot_data[y_var],
        )        

    x_label = get_metric_label(x_var)
    y_label = get_metric_label(y_var)
    plt.xlabel(x_label, size=12)
    plt.ylabel(y_label, size=12)
    plt.title(f"{y_label} by {x_label}", size=14)

### Snapshot of condition (symptoms, behavior)

e.g. to give a doctor a brief overview

In [ ]:
def clean_categorical(s: str) -> str:
    """Create binary from string. Assume missing means 0."""
    if re.match("yes", s, re.I):
        return 1
    if re.match("a bit", s, re.I) or re.match("a little", s, re.I):
        return 1
    # we are only trying to determine if it's an issue, not its magnitude
    if re.match("extreme", s, re.I):
        return 1
    if re.match("^no", s, re.I):
        return 0
    # if we want to ignore missing values
    if s == "unknown":
        return np.nan
    return 0


def generate_summary(
    df: pd.DataFrame,
    start_dt: datetime.datetime,
    end_dt: datetime.datetime,
    categorical_metrics: list[str] = [],
    numeric_metrics: list[str] = [],
    min_pain: int = 3,  # minimum pain (0-10) to consider it worth reporting
) -> str:
    valid = (df.index >= start_dt) & (df.index <= end_dt)
    time_period = df[valid].copy()
    for metric in numeric_metrics:
        time_period[metric] = pd.to_numeric(time_period[metric].astype(str).str.replace(",", ""), errors="coerce")
    
    start_dt_str = start_dt.strftime("%m/%d/%y")
    end_dt_str = end_dt.strftime("%m/%d/%y")
    summary = f"{start_dt_str} - {end_dt_str}\n\n"

    # go in decending order of %
    cat_summaries = []
    for metric in categorical_metrics:
        # Pain metrics are numeric, so we need to create a binary for "pain was > {min_pain}"
        if "pain" in metric:
            time_period[f"{metric}_cat"] = (time_period[metric] >= min_pain).astype(int)
        else:
            time_period[f"{metric}_cat"] = time_period[metric].astype(str).map(clean_categorical)

        pct = round(time_period[f"{metric}_cat"].mean() * 100)
        if metric == "nauseous":
            metric_str = "nausea"
        elif "pain" in metric:
            metric_str = metric.replace(" numeric", "")
            metric_str = f"{metric_str} at least {min_pain} / 10"
        else:
            metric_str = metric

        cat_summaries.append((pct, f"{pct}% of days had {metric_str}."))

    cat_summaries.sort(reverse=True)
    summary += "\n".join([item[1] for item in cat_summaries])
    summary += "\n\n"
    
    for metric in numeric_metrics:
        mean = time_period[metric].mean()
        if np.isnan(mean):
            continue
        # doctors like round numbers
        mean = int(round(mean))

        if metric == "steps":
            mean_str = "{:,}".format(mean)
        elif metric == "previous night Oura score":
            mean_str = f"{mean} / 100"
        elif "pain" in metric:
            mean_str = f"{mean} / 10"
        else:
            mean_str = str(mean)

        if metric in ["BMs", "coffee", "steps"]:
            metric_str = f"{metric} / day"
        elif metric == "previous night Oura score":
            metric_str = "sleep score"
        else:
            metric_str = metric

        res = f"{mean_str} {metric_str}"
        summary += res + "\n"
        
    return summary

### Snapshot of medications

In [ ]:
def generate_med_summary(
    df: pd.DataFrame,
    end_dt: datetime.datetime,
    n_days: int,  # total days to include, including end_dt
) -> str:
    medications_quantified = []
    for unit in DRUG_UNITS:
        medications_quantified.extend([c for c in df.columns if c.endswith(f" {unit}")])
        
    # Remove redundant medications, we already track THC and CBD mg
    medications_quantified = [
        med for med in medications_quantified
        if med not in ["THC/CBD ml", "THC ml"]
    ]

    start_dt = end_dt - datetime.timedelta(n_days - 1)
    valid = (df.index >= start_dt) & (df.index <= end_dt)
    time_period = df[valid].copy()
    # This isn't true if we skipped days
    if len(time_period) < n_days:
        missing_days = n_days - len(time_period)
        print(f"WARNING: missing {missing_days} days.")
    
    start_dt_str = start_dt.strftime("%m/%d/%y")
    end_dt_str = end_dt.strftime("%m/%d/%y")
    if n_days == 1:
        summary = f"{start_dt_str}\n\n"
    else:
        summary = f"{start_dt_str} - {end_dt_str}\n\n"

    for med in medications_quantified:
        mean = int(round(time_period[med].mean()))
        if mean == 0:
            continue
        unit = med.rsplit(" ", 1)[1]
        med_str = med.rsplit(" ", 1)[0]
        summary += f"{mean} {unit} {med_str} / day\n"

    return summary